In [5]:
import pandas as pd

# Load the CSV files
teams_df = pd.read_csv('data/2024/FantasyFnG_2024.csv')
race1_df = pd.read_csv('data/2024/240114_scores_groupbyTier.csv')
race2_df = pd.read_csv('data/2024/240121_scores_groupbyTier.csv')
race3_df = pd.read_csv('data/2024/240211_scores_groupbyTier.csv')
race4_df = pd.read_csv('data/2024/240218_scores_groupbyTier.csv')

In [12]:
# Create a mapping of skier names to points for each race
race1_points = race1_df.set_index('Name')['Points'].to_dict()
race2_points = race2_df.set_index('Name')['Points'].to_dict()
race3_points = race3_df.set_index('Name')['Points'].to_dict()
race4_points = race4_df.set_index('Name')['Points'].to_dict()

In [13]:
# Initialize a list to hold the team scores
team_scores = []
detailed_team_scores = {}

In [22]:
to_update = ["Tier1", "Tier2", "Tier3", "Tier4", "Tier5", "Tier6", "Tier7", "Tier8", "Tier9", "Tier11", "Tier12"]
for col in to_update:
    teams_df[col] = teams_df[col].str.replace('\s+', '', regex=True)
    
teams_df

,Timestamp,Full Name,Member #,Email Address,Fantasy Team Name,Tier1,Tier2,Tier3,Tier4,Tier5,Tier6,Tier7,Tier8,Tier9,Tier10,Tier11,Tier12,Tier13
0,1/11/2024 18:47:40,Brent Norrey,N151,bnorrey@roneymk.com,Set up to win,MikeMcTaggart,JeffCox,VaylenPetrella,MelanieLeistnerLavoie,KevinKilmerChoi,SheriRamshaw,MitchPerreault,RichardFridman,NaN,NaN,GrahamRamshaw,NaN,NaN
1,1/12/2024 12:15:49,Sean Cotnam,C775,sean.cotnam@gmail.com,Rudolph Garmisch Crew,MikeMcTaggart,WilliamCartar,DavidGarber,MarkLaurie,LauraLibby,LisaBugler,TonyPetrella,RachelMegitt,NaN,NaN,HannahKilmerChoi,NaN,NaN
2,1/12/2024 19:14:37,Harvey Libby,S504A,harveylibby@gmail.com,rickeybobbie,MikeMcTaggart,StephanieCoward,RobertHawtin,KevinBeattie,CraigPitman,LisaBugler,MitchPerreault,NaN,PeterLivingston,NaN,NaN,MackenzieHamilton,NaN
3,1/13/2024 13:02:48,Mitch Perreault,C321A,Michelperreaultjr@gmail.com,Perreault_Fantasy,BrandonHune,StephanieCoward,RobertHawtin,MarkLaurie,CraigPitman,JocelynFrench,MitchPerreault,RachelMegitt,NaN,NaN,HannahKilmerChoi,NaN,NaN
4,1/13/2024 13:06:34,Peter Moorhead,M770,peter@timhortons-fergus.com,All Fun little Glory,MikeMcTaggart,StephanieCoward,SeanCotnam,JoannaPerrault,RobReid,EarlDuffy,MitchPerreault,TroyHarrison,NaN,NaN,HannahKilmerChoi,NaN,NaN
5,1/13/2024 16:42:41,Suzanne Wood,W800A,wood.suzanne@gmail.com,Suzanne's Quick Finishers,MikeMcTaggart,WilliamCartar,RobertHawtin,MarkLaurie,LindaLeistner,SheriRamshaw,TusharMehta,MarkSandell,NaN,NaN,HannahKilmerChoi,NaN,NaN
6,1/13/2024 18:50:23,William Cartar,K523C,william.cartar@gmail.com,Will's RealGudPros,MikeMcTaggart,WilliamCartar,SeanCotnam,KevinBeattie,LindaLeistner,ToriAbbott,MitchPerreault,AudunBredur,NaN,NaN,HannahKilmerChoi,NaN,NaN
7,1/13/2024 19:06:17,Carla Nicolson,J910A,carla.nicolson@rogers.com,Red Brick Bruisers,WillStewart,StephanieCoward,DavidGarber,PeterDaly,CraigPitman,LisaBugler,NaN,RachelMegitt,PeterLivingston,NaN,NaN,MackenzieHamilton,NaN
8,1/13/2024 22:36:04,Earl Duffy,D505,earl.duffy@gmail.com,Topshelf,MikeMcTaggart,PeterMoorhead,BradBrock,MarkLaurie,CraigPitman,EarlDuffy,MitchPerreault,EricaReid,NaN,NaN,HannahKilmerChoi,NaN,NaN
9,1/13/2024 22:37:04,Thomas Daly,D565D,t.manski@hotmail.com,Insider Trading,WillStewart,WilliamCartar,RobertSmejkal,MarkLaurie,LauraLibby,JamieHaggart,NaN,TroyHarrison,BrentNorrey,NaN,HannahKilmerChoi,NaN,NaN


In [23]:
# Iterate over each team
for _, team in teams_df.iterrows():
    team_name = team['Fantasy Team Name']
    team_details = {'Total Points': 0, 'Racers': {}}
    
    # Iterate over each tier column to get the skier's name and add their points from each race
    for tier in range(1, 14):
        skier_name = team[f'Tier{tier}']
        if not pd.isna(skier_name):  # Check if skier name is not NaN
            race1_points_skier = race1_points.get(skier_name, 0)
            race2_points_skier = race2_points.get(skier_name, 0)
            race3_points_skier = race3_points.get(skier_name, 0)
            race4_points_skier = race4_points.get(skier_name, 0)
            
            # Add points to the team's total score
            team_details['Total Points'] += race1_points_skier + race2_points_skier + race3_points_skier
            
            # Record the racer's points breakdown
            team_details['Racers'][skier_name] = {
                'Race 1 Points': race1_points_skier,
                'Race 2 Points': race2_points_skier,
                'Race 3 Points': race3_points_skier,
                'Race 4 Points': race4_points_skier,
            }
    
    # Add the team's details to the dictionary
    detailed_team_scores[team_name] = team_details

In [24]:
for team, details in detailed_team_scores.items():
    print(team, "has", details["Total Points"])

Set up to win has 115.0
Rudolph Garmisch Crew has 162.0
rickeybobbie has 143.0
Perreault_Fantasy has 149.0
All Fun little Glory has 139.0
Suzanne's Quick Finishers has 139.0
Will's RealGudPros  has 140.0
Red Brick Bruisers has 151.0
Topshelf  has 162.0
Insider Trading has 174.0
KKC's Ringers has 138.0
EKC's Ringers has 158.0
Rut Riders has 162.0
Slalom on Challenge! has 167.0


In [25]:
# Extract total points and team names from detailed_team_scores for the summary page
summary_data = [{'Team Name': team_name, 'Total Points': details['Total Points']}
                for team_name, details in detailed_team_scores.items()]

# Convert the summary data to a dataframe
summary_df = pd.DataFrame(summary_data)

# Sort the teams by 'Total Points' in descending order
summary_df = summary_df.sort_values(by='Total Points', ascending=False).reset_index(drop=True)

In [28]:
summary_df.to_csv("bar.csv")

In [27]:
# Recreate the Pandas Excel writer
writer = pd.ExcelWriter('data/2024/FantasySkiLeagueResults_Aggregated_240218.xlsx', engine='xlsxwriter')

# Write the summary dataframe to the first sheet
summary_df.to_excel(writer, sheet_name='Summary', index=False)

# Iterate over the detailed scores dictionary and write each team's details to a separate sheet
for team_name, details in detailed_team_scores.items():
    # Convert team_name to string and handle None or NaN values
    team_name_str = 'Unknown Team' if pd.isna(team_name) else str(team_name)
    
    # Create a dataframe from the team's racer details
    racers_df = pd.DataFrame(details['Racers']).T
    racers_df.index.name = 'Racer Name'
    racers_df.reset_index(inplace=True)
    
    # Write the team's dataframe to a sheet named after the team
    racers_df.to_excel(writer, sheet_name=team_name_str[:31], index=False)  # Excel sheet name has max 31 characters

# Close the Pandas Excel writer and save the Excel file
writer.save()